In [1]:
import os
import geopandas as gp
import pandas as pd
import maup
wd = os.getcwd()
raw = os.path.join(wd,'raw-from-source')

# El_Paso_County_CO_Precincts_Population_12_14_2022

## Background:
- We received a request to have total population data on El Paso County, Colorado, precincts.

## Approach:
- Download 2020 PL 94-171 block level data and El Paso County precinct shapefile
- Query data to El Paso County (FIPS 041)
- Aggregate data P0010001 (total population) data from blocks to precincts using the [maup library](https://github.com/mggg/maup)
- Confirm that all data is correctly aggregated

## Links to datasets used:
- [El Paso County, Colorado, Precinct Shapefile (6/1/22)](https://admin.elpasoco.com/free-gis-data/)
- [Colorado block PL 94-171 2020 (by table)](https://redistrictingdatahub.org/dataset/colorado-block-pl-94171-2020-by-table/)

For a full 'raw-from-source' file, contact info@redistrictingdatahub.org

Read in block shapefile with population and precinct shapefile

In [2]:
prec = gp.read_file(os.path.join(raw,'Precinct.shp'))
b = gp.read_file(os.path.join(raw,'co_pl2020_p1_b.shp'))

Query block data to El Paso County, CO (FIPS 041) 

In [3]:
b = b[['P0010001','GEOID20','COUNTY','geometry']]
b['GEOID20'] = b['GEOID20'].astype(str)
b['COUNTY'] = b['COUNTY'].astype(str)
b = b[b['COUNTY']=='041']
display(prec.head())
display(b.head())

,PRECINCT,COM_DIST,SENATE,REP,STATENUM,FEATDATE,POLLCODE,SHAPE_STAr,SHAPE_STLe,geometry
0,644,4,12,21,5.122122e+09,2009-09-01,None,1.065677e+07,17423.556705,"POLYGON ((3229553.461 1329378.609, 3229553.384..."
1,417,2,10,15,5.101521e+09,2002-03-15,None,9.451139e+06,15232.559449,"POLYGON ((3227057.043 1393834.607, 3227079.190..."
2,405,2,10,15,5.101521e+09,2002-03-15,None,1.136839e+07,17327.832633,"POLYGON ((3230247.684 1393968.067, 3230252.120..."
3,643,4,12,21,5.122122e+09,2009-09-01,None,1.202728e+07,21852.789709,"POLYGON ((3233173.230 1327299.421, 3233013.107..."
4,406,2,10,15,5.101521e+09,2005-04-28,None,1.123190e+07,15111.647950,"POLYGON ((3230725.987 1393821.389, 3230912.080..."


,P0010001,GEOID20,COUNTY,geometry
2,21,080410010002024,041,"POLYGON ((-104.82316 38.85724, -104.82314 38.8..."
3,0,080410039114000,041,"POLYGON ((-104.18051 39.12874, -104.17961 39.1..."
4,14,080410074021003,041,"POLYGON ((-104.79440 39.12753, -104.79421 39.1..."
5,55,080410051251008,041,"POLYGON ((-104.68660 38.93196, -104.68639 38.9..."
6,7,080410011012002,041,"POLYGON ((-104.82157 38.87092, -104.82155 38.8..."


Assign blocks to precincts and aggregate total population to precincts

In [4]:
crs = prec.crs
b = b.to_crs(crs)
assignment = maup.assign(b, prec)
prec['TOTPOP20'] = b['P0010001'].groupby(assignment).sum()
prec.head()

C:\Users\SpencerNelson\anaconda3\envs\maup\lib\site-packages\maup\indexed_geometries.py:15: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometry.index = i


,PRECINCT,COM_DIST,SENATE,REP,STATENUM,FEATDATE,POLLCODE,SHAPE_STAr,SHAPE_STLe,geometry,TOTPOP20
0,644,4,12,21,5.122122e+09,2009-09-01,None,1.065677e+07,17423.556705,"POLYGON ((3229553.461 1329378.609, 3229553.384...",2465
1,417,2,10,15,5.101521e+09,2002-03-15,None,9.451139e+06,15232.559449,"POLYGON ((3227057.043 1393834.607, 3227079.190...",2598
2,405,2,10,15,5.101521e+09,2002-03-15,None,1.136839e+07,17327.832633,"POLYGON ((3230247.684 1393968.067, 3230252.120...",2414
3,643,4,12,21,5.122122e+09,2009-09-01,None,1.202728e+07,21852.789709,"POLYGON ((3233173.230 1327299.421, 3233013.107...",2439
4,406,2,10,15,5.101521e+09,2005-04-28,None,1.123190e+07,15111.647950,"POLYGON ((3230725.987 1393821.389, 3230912.080...",2954


Clean file to necessary columns

In [5]:
prec_col_order = ['PRECINCT','TOTPOP20', 'COM_DIST','SENATE', 'REP','geometry']
prec = prec[prec_col_order]
prec

,PRECINCT,TOTPOP20,COM_DIST,SENATE,REP,geometry
0,644,2465,4,12,21,"POLYGON ((3229553.461 1329378.609, 3229553.384..."
1,417,2598,2,10,15,"POLYGON ((3227057.043 1393834.607, 3227079.190..."
2,405,2414,2,10,15,"POLYGON ((3230247.684 1393968.067, 3230252.120..."
3,643,2439,4,12,21,"POLYGON ((3233173.230 1327299.421, 3233013.107..."
4,406,2954,2,10,15,"POLYGON ((3230725.987 1393821.389, 3230912.080..."
...,...,...,...,...,...,...
323,120,1465,5,10,16,"POLYGON ((3198528.806 1398613.019, 3198589.487..."
324,239,2703,1,9,14,"POLYGON ((3203469.840 1411336.160, 3203475.140..."
325,456,884,2,9,15,"POLYGON ((3234987.707 1404415.139, 3234136.392..."
326,526,110,2,35,56,"POLYGON ((3266589.145 1441742.966, 3266529.792..."


Confirm that the total population of blocks in El Paso County match total population of precincts

In [6]:
print(prec['TOTPOP20'].sum())
print(b['P0010001'].sum())

730395
730395


Export data

In [7]:
prec.to_file('./el_paso_county_2022_prec_2020_pop.shp')
prec.drop(columns = 'geometry',inplace=True)
prec.to_csv('./el_paso_county_2022_prec_2020_pop.csv',index=False)